# Tensor to File

In [1]:
import os 
import cv2 
import glob 
import torch
import numpy as np 
from tqdm import tqdm
from PIL import Image  
import io
from data.torch_dataset import Transform 

def read_pfm(file : str):
    with open(file, 'rb') as f:
        # Header
        header = f.readline().decode('utf-8').rstrip()
        color = header == 'PF'

        # Dimensions
        dim_line = f.readline().decode('utf-8').rstrip()
        width, height = map(int, dim_line.split())

        # Scale (endian and range)
        scale = float(f.readline().decode('utf-8').rstrip())
        endian = '<' if scale < 0 else '>'  # Little endian if scale is negative

        # Pixel data
        data = np.fromfile(f, endian + 'f')  # Read floats from file
        shape = (height, width, 3) if color else (height, width)

        # Reshape and flip the image vertically
        return np.reshape(data, shape)[::-1, ...]
        
def min_max_scaler(tensor):
    # Hitung nilai minimum dan maksimum dari tensor input
    tensor_min = tensor.min()
    tensor_max = tensor.max()

    # Terapkan Min-Max scaling
    scaled_tensor = (tensor - tensor_min) / (tensor_max - tensor_min)  # Scaling ke [0, 1]
    return scaled_tensor

In [2]:
dataset_root = "/mnt/d/backup/InSpaceType_all_sp8/home/InSpaceType" 
dataset = []
for seq in os.listdir(dataset_root): 
    sequence = os.path.join(dataset_root, seq) 

    images = sorted(glob.glob(sequence + "/*.jpg"))
    pmfs = sorted(glob.glob(sequence + "/*.pfm"))  

    for image, pmf in zip(images, pmfs): 
        dataset.append({'image':image, 'label':pmf})

In [ ]:
image_tensors = [] 
pfm_tensors = []
for dat in tqdm(dataset): 
    image = dat["image"]
    image = Image.open(image)
    transformed = Transform.train_transform(image)
    image_tensors.append(transformed)

    label = dat["label"]
    pfm_image = read_pfm(label) 
    pfm_image = cv2.resize(pfm_image, (256,256))
    pfm_tensor = torch.from_numpy(pfm_image)
    pfm_tensor = min_max_scaler(pfm_tensor) 
    pfm_tensors.append(pfm_tensor) 


In [4]:
torch.save(image_tensors, 'images_tensor.pt')
torch.save(pfm_tensors, 'labels_tensor.pt')

# Coba

In [2]:
import torch 
from torch.utils.data import DataLoader
from data.torch_dataset import DepthDataset 
import matplotlib.pyplot as plt

In [3]:
images_tensor = torch.load("images_tensor.pt") 
labels_tensor = torch.load("labels_tensor.pt") 

/tmp/ipykernel_46652/911462910.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  images_tensor = torch.load("images_tensor.pt")
/tmp/ipykernel_46652/911462910.py:2: Future

# Debug

In [4]:
from sklearn.model_selection import train_test_split 
from torchmetrics.image import StructuralSimilarityIndexMeasure 

ssim = StructuralSimilarityIndexMeasure(data_range=1.0)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    images_tensor, labels_tensor, 
    test_size=0.2, random_state=42
)
train_dataset = DepthDataset(X_train, y_train)
val_dataset = DepthDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [11]:
for x, y in val_loader: 
    y = y.unsqueeze(1)

    loss = ssim(y,y) 
    
    if torch.isnan(loss).any() or torch.isinf(loss).any():
        print("wkwkwkw")
